In [2]:
from sklearn.metrics.pairwise import euclidean_distances,manhattan_distances, cosine_distances
import pandas as pd


In [3]:
# method untuk mencari tetangga terdekat
def tetangga_terdekat(k):
    neigbor = {}
    near = ('', 0)
    # near = (value terjadi, tetangganya)
    for i in k:
        if i in neigbor:
            neigbor[i] += 1
        else:
            neigbor[i] = 1
        # menyimpan tetangga sebanyak k
        if neigbor[i] > near[1]:
            near = (i, neigbor[i])
    return near

In [4]:
# menghitung jarak euclidean
def jarak_euclidean(x, y):
        return euclidean_distances(x,y)

In [5]:
# fungsi S untuk Validity
def fungsi_S(a, b):
    if a == b:
        return True
    return False

In [6]:
def validity(distance, y, k):
    v_arr = []
    current_index = 0
    n_val = open('output/neigbor.txt', 'w')
    for i in distance:
        sorted_index = sorted(range(len(i)), key=lambda k: i[k])
        fungsi_k = []
        for j in range(k):
            fungsi_k.append(y[sorted_index[j + 1]])

        label = 0
        print('{} = neigbor: {}'.format(y[current_index], fungsi_k))
        n_val.write('{} = neigbor: {}'.format(y[current_index], fungsi_k) + '\n')
    
        #st.write('{}, neigbor: {}'.format(y[current_index], fungsi_k))
        for w in fungsi_k:
            if fungsi_S(y[current_index], w):
                label += 1
        validity_result = 1 / k * label
        v_arr.append(validity_result)
        current_index += 1
    n_val.close()
    return v_arr

In [14]:

class ModifiedKNN(object):
    def __init__(self, k=3):
        self.k = k

    # fit data x train dan data y train
    def fit(self, X, y):
        self.X_train = X

        if isinstance(y, pd.Series):
            self.y = y.values.ravel()
        else:
            self.y = y

        self.distance = jarak_euclidean(X, X)
        self.validity = validity(self.distance, self.y, self.k)

    def predict(self, X_test):
        if isinstance(X_test, pd.Series):
            test = X_test.values
        else:
            test = X_test

        pred_label = []
        distances = jarak_euclidean(X_test, self.X_train)
        #print(distances)

        #Hitung Weighted Voting
        for i in distances:
            weight_voting = []
            for j in range(len(self.validity)):
                weight = self.validity[j] * (1 / (i[j] + 0.5))
                weight_voting.append(weight)
            sorted_index = sorted(range(len(weight_voting)), key=lambda k: weight_voting[k], reverse = True)
            mknn_label = []
            y = self.y
            for w in range(self.k):
                mknn_label.append(y[sorted_index[w]])

            neigbor,count = tetangga_terdekat(mknn_label)
            #print(mknn_label)

            pred_label.append(neigbor)

        return pred_label, distances

# Test Run

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, accuracy_score
import numpy as np
from heapq import nsmallest as nMin

In [9]:
df = pd.read_csv('output/Clean_text_Sentiment.csv')
X = df['text']
y = df['Sentiment']

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

tf = TfidfVectorizer()
enc = LabelEncoder()
X_train = tf.fit_transform(X_train)
X_test = tf.transform(X_test)
y_train = enc.fit_transform(y_train)
nilai_k = 3
clf = ModifiedKNN(k=nilai_k)
clf.fit(X_train, y_train)
predict, jarak =  clf.predict(X_test)
print(X_train)

y_test = y_test.apply(lambda x: 1 if 'Positive' else 0)
acc = accuracy_score(y_test, predict)*100
print("ACC :",acc)

#print(predict)
print("XTrain :",X_train.shape);print("XTest",X_test.shape);print("yTrain",y_train.shape);print("ytest:",y_test.shape)
#jarak = [x for l in jarak for x in l]
jarak = [nMin(nilai_k,map(float,i)) for i in jarak]
    
with open('jarak tetangga.txt', 'w') as f, open('label pred.txt', 'w') as g:
    f.write('\n'.join(str(ls) for ls in jarak))
    g.write(str(predict))

print('\n'.join(str(el) for el in jarak))

0 = neigbor: [0, 0, 0]
0 = neigbor: [0, 0, 0]
0 = neigbor: [0, 0, 0]
1 = neigbor: [1, 0, 1]
0 = neigbor: [0, 0, 0]
0 = neigbor: [0, 0, 0]
0 = neigbor: [0, 1, 1]
1 = neigbor: [0, 0, 1]
0 = neigbor: [0, 0, 0]
0 = neigbor: [0, 1, 0]
0 = neigbor: [0, 0, 1]
1 = neigbor: [0, 1, 0]
1 = neigbor: [0, 0, 1]
0 = neigbor: [0, 0, 0]
1 = neigbor: [0, 0, 0]
0 = neigbor: [0, 0, 1]
0 = neigbor: [0, 0, 0]
0 = neigbor: [0, 0, 0]
0 = neigbor: [0, 0, 0]
0 = neigbor: [0, 0, 0]
0 = neigbor: [1, 1, 1]
0 = neigbor: [0, 0, 0]
0 = neigbor: [0, 0, 0]
0 = neigbor: [0, 0, 0]
0 = neigbor: [0, 0, 1]
0 = neigbor: [0, 0, 0]
1 = neigbor: [1, 0, 0]
0 = neigbor: [0, 0, 1]
0 = neigbor: [0, 0, 0]
0 = neigbor: [0, 0, 0]
0 = neigbor: [1, 0, 0]
0 = neigbor: [0, 0, 0]
1 = neigbor: [0, 1, 1]
1 = neigbor: [0, 0, 0]
0 = neigbor: [0, 0, 0]
1 = neigbor: [0, 0, 0]
1 = neigbor: [1, 0, 0]
0 = neigbor: [0, 0, 0]
0 = neigbor: [0, 0, 1]
0 = neigbor: [0, 1, 1]
  (0, 70)	0.28782756785913277
  (0, 263)	0.9576822495902785
  (1, 67)	0.41736118

In [33]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

l = knn.kneighbors( n_neighbors=3, return_distance=False)
print(l)

[[13 15 25]
 [31 15 38]
 [15 13  0]
 [33 20 36]
 [22  6 24]
 [ 6 24 15]
 [29 14 33]
 [15 10 32]
 [27 31 13]
 [15 33 39]
 [31 15 14]
 [15 32 25]
 [27 15 26]
 [ 0 15 10]
 [ 6 24 31]
 [13  0 33]
 [15 18 34]
 [15 13  0]
 [15 16 13]
 [27 15 13]
 [33 36  3]
 [ 6 24 15]
 [ 4 15 13]
 [25  6 24]
 [ 6 15 14]
 [ 0 23 13]
 [12 15  8]
 [19  8 12]
 [31  1 34]
 [ 6 15 24]
 [35 31 15]
 [ 1  6 24]
 [15 11  7]
 [ 6 15 20]
 [13 15 28]
 [30  6 24]
 [33 20 31]
 [15 13  0]
 [15  1  7]
 [15 33  7]]
